This Notebook launches a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).
 The calculations can be very long. 
 In order to use a HPC cluster ``..\.BoSSS\etc\BatchProcessorConfig.json`` must be modified for the execution queue

In [ ]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [ ]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Get Default Queue

In [ ]:
BoSSSshell.ExecutionQueues

default database path

In [ ]:
BoSSSshell.WorkflowMgm.DefaultDatabase.Path

list of available queues

In [ ]:
BoSSSshell.GetDefaultQueue()

Here we prescribe all combinations we want to study

In [ ]:
using XESF.Fluxes;
using ApplicationWithIDT;

int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {1,2,4};
double[] Aggs = new double[] {0.4};
var iFluxes = new int[] {0}; //{ ConvectiveInterfaceFluxes.GodunovInterface,ConvectiveInterfaceFluxes.RoeInterface, ConvectiveInterfaceFluxes.CentralFluxInterface, ConvectiveInterfaceFluxes.OptimizedHLLCInterface };
var iProbs= new int[] {0}; //{ OptProblemType.FullEnRes, OptProblemType.EnResOnlyNearBand, OptProblemType.RankineHugoniot};
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;  


get data we need to initialize the initial guess for the shock LevelSet

In [ ]:
var bowShockPoints =new Tuple<byte[],string>( System.IO.File.ReadAllBytes("BowShockPoints.txt"),"BowShockPoints.txt");
var LevelSetDB =new Tuple<byte[],string>(System.IO.File.ReadAllBytes("bosss_db_levelSets.zip"),"bosss_db_levelSets.zip");

run the study

In [ ]:
foreach(int iProb in iProbs) 
foreach(int iflux in iFluxes) {
foreach(double agg in Aggs) {
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
                var ctrl_temp=XESF.XESFHardCodedControl.XDGBS_Cluster(gammaMax: 1.0 ,DegE: p ,numX: 5*n ,numY: 16*n ,agg: agg ,plotInterval: -1 ,iProb: iProb ,iflux: iflux ,aRI:true,dbPath:  dbPath  );
                Job j = new Job(ctrl_temp.SessionName,new XESF.XESFMain().GetType());// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.SetControlStatement($"cs:XESF.XESFHardCodedControl.XDGBS_Cluster(gammaMax:{1.0},DegE:{p},numX:{5*n},numY:{16*n},agg:{agg},plotInterval:{-1},iProb:{iProb},iflux:{iflux},aRI:true,dbPath:@\"{dbPath}\")");
                j.AdditionalDeploymentFiles.Add(bowShockPoints);
                j.AdditionalDeploymentFiles.Add(LevelSetDB);
                j.NumberOfThreads=1;
                
                Console.WriteLine("!!!!!!!!!!!!!!!! WARNING: OPENMP Paralelization turned off !!!!!!!!!!!!");
                j.Activate(BoSSSshell.ExecutionQueues[0]); 
        };                                                                                                                                                          
    };
    }
};


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(7*86400 , 60); // wait for one week, or until all jobs finish

## Check if residual norms converged

Load sessions

In [ ]:
var sessions = BoSSSshell.WorkflowMgm.DefaultDatabase.Sessions.Where(si=> si.Name.Contains("iFlx0"));
sessions

Obtain the residual norms

In [ ]:
var ass= new MatrixAssembler(1);
var res = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).ResHistory.Last());
res

In [ ]:
var ass= new MatrixAssembler(1);
var enres = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).EnResHistory.Last());
enres

Assert if smaller than threshold

In [ ]:
res.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1));
enres.ForEach(r => NUnit.Framework.Assert.IsTrue(r<50));